In [30]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import random
import glob
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
from tensorflow.keras import applications
from tensorflow.keras import callbacks
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras import callbacks

DATADIR = "C:/Users/Kajoj/magisterka/dataset"
CATEGORIES = ["benign", "malignant"]
IMG_SIZE = 200
CALLBACK = callbacks.EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=15, restore_best_weights=True)


In [2]:
def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

In [3]:
training_data = []
create_training_data()
print(len(training_data))
random.shuffle(training_data)

12210


In [4]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3) #if it wasnt gray scale then 3 instead of 1
y = np.array(y)

X= X/255.0

In [5]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [2]:
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.30)

In [31]:
model = tf.keras.Sequential([
    tf.keras.applications.DenseNet121(weights=None, input_shape=X.shape[1:]),
    tf.keras.layers.Flatten(name="flatten"),
    tf.keras.layers.Dense(256, activation="relu", name="dense1"),
    tf.keras.layers.BatchNormalization(name="norm1"),
    tf.keras.layers.Dense(256, activation=None, name="dense4"),
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1), name="output1")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())



In [32]:

history = model.fit(
    X_train, Y_train, batch_size=32,
    epochs=150, validation_split = 0.3, callbacks=CALLBACK)

Epoch 1/150
187/187 [==============================] - 990s 5s/step - loss: 0.7161 - val_loss: 0.9113
Epoch 2/150
187/187 [==============================] - 958s 5s/step - loss: 0.6658 - val_loss: 1.0795
Epoch 3/150
187/187 [==============================] - 959s 5s/step - loss: 0.6548 - val_loss: 1.0000
Epoch 4/150
187/187 [==============================] - 956s 5s/step - loss: 0.6132 - val_loss: 0.6053
Epoch 5/150
187/187 [==============================] - 958s 5s/step - loss: 0.6071 - val_loss: 0.8113
Epoch 6/150
187/187 [==============================] - 954s 5s/step - loss: 0.5617 - val_loss: 0.7525
Epoch 7/150
187/187 [==============================] - 954s 5s/step - loss: 0.5233 - val_loss: 1.0040
Epoch 8/150
187/187 [==============================] - 951s 5s/step - loss: 0.5261 - val_loss: 0.6470
Epoch 9/150
187/187 [==============================] - 951s 5s/step - loss: 0.5501 - val_loss: 0.5848
Epoch 10/150
187/187 [==============================] - 953s 5s/step - loss: 0.522

In [33]:
train_embs = model.predict(X_train)
test_embs = model.predict(X_test)

115/115 [==============================] - 113s 981ms/step


In [38]:
import io
import tensorflow_datasets as tfds

np.savetxt("vecs-densenet121-d256-d256.tsv", test_embs, delimiter='\t')

out_m = io.open('meta-densenet121-d256-d256.tsv', 'w', encoding='utf-8')
[out_m.write(str(x) + "\n") for x in Y_test]
out_m.close()


try:
  from google.colab import files
  files.download('vecs-densenet121-d256-d256.tsv')
  files.download('meta-densenet121-d256-d256.tsv')
except:
  pass

In [40]:
import pickle

pickle_out = open("X_train.pickle", "wb")
pickle.dump(X_train, pickle_out)
pickle_out.close()

pickle_out = open("Y_train.pickle", "wb")
pickle.dump(Y_train, pickle_out)
pickle_out.close()

pickle_out = open("X_test.pickle", "wb")
pickle.dump(X_test, pickle_out)
pickle_out.close()

pickle_out = open("Y_test.pickle", "wb")
pickle.dump(Y_test, pickle_out)
pickle_out.close()

In [41]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(train_embs, Y_train)
y_test_pred_knn = knn.predict_proba(test_embs)


In [42]:
predict = []
for pred in y_test_pred_knn:
    if(pred[0]>=pred[1]):
        predict.append(0)
    else:
        predict.append(1)

In [43]:
TP = 0
FP = 0
TN = 0
FN = 0

for i in range (len(predict)):
    if predict[i] == 0 and Y_test[i] == 0:
        TN += 1
    elif predict[i] == 1 and Y_test[i] == 1:
        TP += 1
    elif predict[i] == 0 and Y_test[i] == 1:
        FN += 1
    elif predict[i] == 1 and Y_test[i] == 0:
        FP += 1

In [44]:
czulosc = TP / (TP + FN)
swoistosc = TN / (TN + FP)

print(czulosc)
print(swoistosc)

0.9137554585152838
0.9623156744948116


In [45]:
model.save('densenet121-d256-d256-2/densenet121-d256-d256')

INFO:tensorflow:Assets written to: densenet121-d256-d256-2/densenet121-d256-d256\assets


INFO:tensorflow:Assets written to: densenet121-d256-d256-2/densenet121-d256-d256\assets
